In [1]:
# ==============================================================================
# BLOCK 1: INSTALL LIBRARIES
# ==============================================================================
!pip install fastapi uvicorn pyngrok pydantic torch transformers emoji -q
print("✅ Libraries installed.")

# ==============================================================================
# BLOCK 2: THE API SERVER CODE
# ==============================================================================
import uvicorn
import re
import emoji
import torch
import torch.nn.functional as F
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import RobertaTokenizer, RobertaModel
from torch.nn import Module, Linear
from pyngrok import ngrok
import threading
import os
# This is the NEW part for using Kaggle Secrets
from kaggle_secrets import UserSecretsClient

# --- Define the request body for the API ---
class Tweet(BaseModel):
    text: str

# --- Preprocessing Function (must be identical to the one used for training) ---
def preprocess_text(text: str):
    if not isinstance(text, str): return ""
    text = emoji.demojize(text)
    text = text.lower()
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- Re-define the Model Architecture (must be identical to your trained model) ---
class EmoRoBERTa(Module):
    def __init__(self, n_emotions):
        super(EmoRoBERTa, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.emotion_classifier = Linear(self.roberta.config.hidden_size, n_emotions)
        self.intensity_regressor = Linear(self.roberta.config.hidden_size, n_emotions)
        self.sarcasm_detector = Linear(self.roberta.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        return (self.emotion_classifier(outputs), self.intensity_regressor(outputs), self.sarcasm_detector(outputs))

# --- Load Model, Tokenizer, and Configuration ---
model = None
try:
    # ** FIX #1: PASTE YOUR CORRECT FILE PATH HERE **
    MODEL_PATH = '/kaggle/input/emoberta_model/pytorch/default/1/emoberta_model.bin' # <-- PASTE CORRECT PATH

    # This list is CORRECT because it will have the 12 emotions from your original training
    EMOTION_CLASSES = ['anger', 'annoyance', 'approval', 'caring', 'curiosity', 'fear', 'gratitude', 'joy', 'love', 'neutral', 'sadness', 'surprise'] # <-- PASTE YOUR REAL LIST HERE
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = EmoRoBERTa(n_emotions=len(EMOTION_CLASSES)).to(device)
    
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        model.eval()
        print(f"✅ Model loaded successfully from {MODEL_PATH}!")
    else:
        print(f"❌ ERROR: Model file not found at {MODEL_PATH}")
        model = None
        
except Exception as e:
    print(f"--- ❌ An error occurred while loading the model: {e} ---")
    model = None

# --- Initialize the FastAPI app ---
app = FastAPI()

@app.get("/")
def read_root():
    return {"status": "EmoSense API is running"}

@app.post("/analyze")
def analyze_emotion(tweet: Tweet):
    if not model:
        return {"error": "Model is not loaded. Please check the logs."}
    # ... (rest of the function is the same)
    text = tweet.text
    cleaned_text = preprocess_text(text)
    encoding = tokenizer.encode_plus(
        cleaned_text, add_special_tokens=True, max_length=128, return_token_type_ids=False,
        padding='max_length', truncation=True, return_attention_mask=True, return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        emotion_logits, intensity_scores, sarcasm_logits = model(input_ids, attention_mask)
        emotion_probs = F.sigmoid(emotion_logits).flatten().cpu().numpy()
        sarcasm_prob = F.sigmoid(sarcasm_logits).flatten().cpu().numpy()[0]
        intensity_scores = intensity_scores.flatten().cpu().numpy()
    return {
        "text": text,
        "emotions_confidence": {emotion: float(prob) for emotion, prob in zip(EMOTION_CLASSES, emotion_probs)},
        "predicted_intensity": {emotion: float(score) for emotion, score in zip(EMOTION_CLASSES, intensity_scores)},
        "sarcasm_score": float(sarcasm_prob)
    }
    
# ==============================================================================
# BLOCK 3: LAUNCH THE SERVER AND GET YOUR PUBLIC URL
# ==============================================================================
# ** FIX #2: ADDED CODE TO USE YOUR KAGGLE SECRET **
try:
    # This line securely gets your authtoken from the Kaggle Secret you created
    ngrok.set_auth_token(UserSecretsClient().get_secret("NGROK_AUTH_TOKEN"))
    print("✅ ngrok token authenticated successfully.")
except Exception as e:
    print(f"❌ Could not authenticate ngrok. Error: {e}")
    print("Please ensure you've added a Kaggle Secret with the label 'NGROK_AUTH_TOKEN'.")

# Run the app and get the public URL
public_url = ngrok.connect(8000)
print(f"🚀 EmoSense API is live! Your public URL is: {public_url}")

# Start the server
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 29.6 MB/s eta 0:00:00
ERROR: pip's dependency

2025-10-18 06:13:55.636200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760768035.863022      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760768035.921227      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded successfully from /kaggle/input/emoberta_model/pytorch/default/1/emoberta_model.bin!
✅ ngrok token authenticated successfully.                                                           
🚀 EmoSense API is live! Your public URL is: NgrokTunnel: "https://exogenously-connectable-li.ngrok-free.dev" -> "http://localhost:8000"
